## Azure AI Foundry 에이전트

MAF(Microsoft Agent Framework)는 Azure AI Foundry Agent 서비스를 사용하는 에이전트를 만드는 것도 지원합니다. 서비스에서 자체 제공하는 대화 스레드를 활용해서 영구적인 서비스-기반의 에이전트 인스턴스를 만들 수 있습니다.

#### 환경 변수  

Azure AI Foundry Agen를 사용하기 전에 다음 환경 변수를 설정해야 합니다. 또는, 코드에서 직접 지정해도 됩니다.   
다만, [0.Prerequisite.ipynb - 사전 준비 사항 및 환경 설정](0.Prerequisite.ipynb) 단계에서 이미 .env 파일에 설정했다면, 여기서 별도로 설정할 필요는 없습니다.

> ``` bash
> export AZURE_AI_PROJECT_ENDPOINT="https://<your-project>.services.ai.azure.com/api/projects/<project-id>"
> export AZURE_AI_MODEL_DEPLOYMENT_NAME="gpt-4o-mini"
>```

#### 설치

Agent Framework의 Azure AI 패키지를 프로젝트에 추가합니다

In [ ]:
# 안정성을 위해 검증된 버전 사용 (2026-01-16 릴리스)
%pip install agent-framework-azure-ai==1.0.0b260116

# 참고: 현재 MAF는 프리뷰 상태로 빠르게 발전 중
# - 1.0.0b260114: 대규모 Breaking Changes 포함 (피하는 것이 좋음)
# - 1.0.0b260116: 상대적으로 안정적, 주요 버그 수정 포함
# - 1.0.0b260123/260127: 최신 버전이나 아직 검증이 덜 됨
# 
# GitHub 릴리스 노트: https://github.com/microsoft/agent-framework/releases

### Azure AI Foundry 에이전트

#### 기본 에이전트 생성하기

가장 빠르게 에이전트를 생성하는 방법은 AzureAIAgentClient를 사용하는 것입니다. 환경 변수를 사용하여 에이전트를 생성할 수 있습니다. 단, 이 에이전트는 영구적이지 않으며, 세션이 종료되면 사라집니다.

In [ ]:
import asyncio
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

async def main():
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            credential=credential,
            agent_name="HelperAgent"
        ).as_agent(
            instructions="You are a helpful assistant."
        ) as agent,
    ):
        result = await agent.run("Hello!")
        print(result.text)

await main()

### Foundry에 유지되는 에이전트 생성 및 관리 

에이전트 수명주기를 더 제어하기 위해서는, Azure AI Projects 클라이언트를 사용하여 Foundry에 유지되는 에이전트를 생성합니다. 다음의 코드는 현재 Fodunry 프로젝트에 존재하는 에이전트 목록을 나열하고, 지정한 이름의 에이전트가 없으면 새로 생성하는 예제입니다. 

In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient

# AgentsClient를 사용하여 Persistent Agent 생성
# azure-ai-projects 2.0.0b3에서는 문제가 있는 것으로 보임.
# 그래서, MAF의 일부인 azure.ai.projects.AIProjectClient 대신 
# azure.ai.agents.AgentsClient를 사용해서 영구적인 에이전트 생성
credential = DefaultAzureCredential()
agents_client = AgentsClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=credential
)

#원래의 코드는 다음과 같음
# async with (
#     AzureCliCredential() as credential,
#     AIProjectClient(
#         endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
#         credential=credential
#     ) as project_client,
# ):
#     # Create a persistent agent
#     created_agent = await project_client.agents.create_agent(
#         model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
#         name="PersistentAgent",
#         instructions="You are a helpful assistant."
#     )

#새로 생성할 에이전트 이름
agent_name="PersistentAgent"

# 기존 에이전트 목록 조회
print("=== 기존 에이전트 목록 ===")
for agent in agents_client.list_agents():
    print(f"Agent ID: {agent.id}, Name: {agent.name}")

# 기존 Agent가 없으면 새로운 Agent 생성
agent = None
for existing in agents_client.list_agents():
    if existing.name == agent_name:
        agent = existing
        break

if(agent):
    print(f"에이전트 '{agent_name}'(이)가 이미 존재합니다. 새로 생성하지 않습니다.")
else:
    print(f"에이전트 '{agent_name}'(이)가 존재하지 않습니다. 새로 생성합니다.")
    agent = agents_client.create_agent(
        model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
        name=agent_name,
        instructions="You are a helpful assistant who speak in korean"
    )

print(f"에이전트 ID: {agent.id}")
print(f"에이전트 이름: {agent.name}")
print(f"모델: {agent.model}")
print(f"instruction : {agent.instructions}")

# 기존(또는 생성된) 에이전트를 사용하여 ChatAgent 구성
chat_agent = ChatAgent(
    chat_client=AzureAIAgentClient(
        credential=credential,
        agent_id=agent.id
    )
)
result = await chat_agent.run("오늘도 좋은 하루!")
print(result.text)

agent_id=agent.id
# 정리 (필요시 주석 해제)
# agents_client.delete_agent(agent.id)
# print(f"\n에이전트 삭제 완료: {agent.id}")

#### 기존 Azure AI Foundry 에이전트 사용하기

만일 Azure AI Foundry 프로젝트에 이미 에이전트가 존재한다면, 그 에이전트의 ID를 지정하여 기존 에이전트의 인스턴스를 만들고 사용할 수도 있습니다. 다음 예제에서는 이전에 생성된 에이전트의 ID를 사용하여 에이전트를 불러오고, 질문을 던지는 방법을 보여줍니다.

In [ ]:
import asyncio
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

agent_id="<이전의 agent id>"  # 이전에 생성된 에이전트를 여기에 할당
agent_id="asst_dwy8NMHcX8g0Id5w4KQssTZM"
async def main():
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(
                credential=credential,
                agent_id=agent_id
            )
        ) as agent,
    ):
        
        result = await agent.run("한국은 어디에 있나요?")
        print(result.text)

await main()